In [ ]:
%%capture
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from lfimodels.ricker.Ricker import Ricker
from lfimodels.ricker.RickerStats import RickerStats

# LFI for Ricker model

http://www.maths.lth.se/matstat/staff/umberto/SyntheticLikelihoods_MSCproject/wood_2010.pdf

In [ ]:
n_params = 3

# problem setup
T, burn_in = 50, 100
thetao = np.array([3.8, 0.3, 10.])
xo = {'data' : np.array([
    0,   0,   4,  81,   0,  27,  52,  19, 134,   0,
    0,  27,  84,   1, 166,   0,   1,   0,  37,  65,
    4,  50,   4, 121,   0,   0,  38,  36,  36,  55,
   19, 155,   0,   0,   0,  59,  18, 128,   0,   1,
   18, 138,   0,   0,  10, 148,   0,   0,   0,  22], dtype=np.int64) }
s = RickerStats(obs = xo['data'])
obs_stats = s.calc([xo])

# define generator init (new for every seed)
log_sig = False
log_phi = False
lthetao = thetao.copy()
if log_sig:
    lthetao[1] = -np.log(thetao[1])
if log_phi:
    lthetao[2] = -np.log(thetao[2])
    
def init_g(seed, log_sig=log_sig, log_phi=log_phi):
    m = Ricker(dim=n_params, 
               burnIn=burn_in, 
               T=T, 
               log_r0=0, 
               log_sig=log_sig,
               log_phi=log_phi,
               seed=seed)
    if log_sig and log_phi:        
        p = dd.StackedDistribution([
                dd.Uniform(lower=np.array([3.0]), upper=np.array([5.0]), seed=seed),
                dd.Gamma(alpha=1., beta=1., offset = -np.log(0.8), seed=seed+1),
                dd.Gamma(alpha=1., beta=1., offset = -np.log(20.), seed=seed+2)])
    elif log_sig:        
        p = dd.StackedDistribution([
                dd.Uniform(lower=np.array([3.0]), upper=np.array([5.0]), seed=seed),
                dd.Gamma(alpha=1., beta=1., offset = -np.log(0.8), seed=seed+1),
                dd.Uniform(lower=np.array([4.0]), upper=np.array([20.0]), seed=seed+2)])
    else:
        p = dd.Uniform(lower=np.array([3.0, 0.0,  4.0]), 
                       upper=np.array([5.0, 0.8, 20.0]),
                       seed=seed)
    return dg.Default(model=m, prior=p, summary=s)

plt.figure(figsize=(16,8))
x_range = np.arange(xo['data'].size).reshape(-1, 1) + 1
plt.plot(x_range, xo['data'], 'bo')
plt.plot(np.hstack([x_range,x_range]).T, np.hstack([np.zeros_like(x_range), xo['data'].reshape(-1,1)]).T, 'b')
plt.title('observed data (not summary stats!)')
plt.xlabel('time bin t')
plt.ylabel('observed count')
plt.axis([0, 51, 0, 180])
plt.show()

g = init_g(seed=42, log_sig=False, log_phi=False)
_, pilots =g.gen(10000)
stats_mean, stats_std = pilots.mean(axis=0).reshape(1,13), pilots.std(axis=0).reshape(1,13)

class normed_summary():
    
    def calc(self, y):

        x = RickerStats(obs = xo['data']).calc(y)

        return (x-stats_mean)/stats_std
    
g.summary = normed_summary()
obs_statz = (obs_stats-stats_mean)/stats_std
th, x = g.gen(1000)

dx = np.linalg.norm(x-obs_statz, axis=1)
plt.hist(dx, np.linspace(dx.min(), dx.max(), 25))
plt.xlabel('| x - xo |')
plt.ylabel('rel. freq')
plt.title('standardized summary statistics')
plt.show()


def plot_res(out):

    plt.figure(figsize=(12,12))
    axes = [ [3., 5.],
             [0., 0.8],
             [4., 20.] ]
    data_path = 'results/'
    tmp = np.load(data_path + 'Ricker_rejABC_xoGutmann_1e8simulations_results.npy')[()]
    th_post = tmp['samples']
    iws = 1./th_post.shape[0] * np.ones(th_post.shape[0])
    del tmp


    for j in range(3):    
        plt.subplot(3,3,j+1)
        plt.hist(out[0][-1][:,j], 
                 bins=np.linspace(axes[j][0], axes[j][1], 50), 
                 weights= np.exp(out[2][-1]),
                 normed=True)

        plt.subplot(3,3,j+4)
        plt.hist(out[0][-1][:,j], 
                 bins=np.linspace(axes[j][0], axes[j][1], 50), 
                 normed=True)    

        plt.subplot(3,3,j+7)
        plt.hist(th_post[:,j], 
                 bins=np.linspace(axes[j][0], axes[j][1], 50),  
                 weights=iws, 
                 normed=True, 
                 facecolor=np.array([80, 160, 220])/256)

    plt.show()


In [ ]:
from lfimodels.abc_methods.run_abc import run_smc


seeds = range(42, 52)

algos = ['raw', 'AW', 'RA', 'OW']
all_raw, all_AW, all_RA, all_OW = [], [], [], []
all_ = [all_raw, all_AW, all_RA, all_OW]

for seed in seeds:
    for a in range(len(algos)):
                
        g=init_g(seed=seed, log_sig=False, log_phi=False)
        kwargs = {'model' : g.model,
                  'prior' : g.prior,
                  'summary' : normed_summary(), 
                  'obs_stats': obs_statz.copy(),
                  'n_particles' : 500, 
                  'eps_init' : [2, 1.5, 1.0, 0.75, 0.6, 0.5], 
                  'maxsim' : 1e4,
                  'fn' : None,
                  'seed': seed}
        
        AW     = True if algos[a] in ['AW', 'RA', 'OW'] else False
        regAdj = True if algos[a] in ['RA', 'OW'] else False
        OW     = True if algos[a] in ['OW'] else False
        
        out = run_smc(**kwargs, 
                      AW=AW, 
                      regAdj=regAdj, 
                      OW=OW)
        plot_res(out)
        all_[a].append(out)

In [ ]:
from delfi.utils.viz import plot_pdf
from scipy.stats import lognorm


def run_panel_c(algo, seed, rounds=5, n_train=[200, 1000]):

    kwargs = {'generator': init_g(seed, log_sig=False, log_phi=False), 
              'reg_lambda': 0.0,
              'init_norm': True,
              'prior_norm': True,
              'reinit_weights' : True,
              'pilot_samples': 1000,
              'n_components': 3, 
              'n_hiddens': [20,20], 
              'verbose' : True,
              'obs': obs_stats.copy(),
              'seed': seed}

    # setup of learning schedule
    train = []
    for r in range(rounds):
        train.append(n_train[0])
    train[-1] = n_train[-1]
    round_cl=999
    minibatch=50
    n_gradients = 100000
    epochs = [n_gradients // (n_train//minibatch) for n_train in train]
    print('n_train :', train)
    print('epochs :', epochs)

    cbk_feature_layer = 0
    
    
    plogr = dd.Gaussian(m=[4], S=[[0.1]], seed=seed)
    psigma = dd.Uniform(lower=[0], upper=[0.8], seed=seed+1)
    pphi = dd.Gaussian(m=[10.], S=[[1.5]], seed=seed+2)
    prop = dd.StackedDistribution([plogr, psigma, pphi])
    
    
    if algo == 'CDELFI':
        inf = infer.CDELFI(**kwargs)
        inf.generator.proposal = prop
        log_, train_data_, posteriors_ = inf.run(n_train=train,
                                              minibatch=minibatch,         
                                              n_rounds=len(train),
                                              epochs=epochs)
        log,train_data,posteriors = [None,*log_],[None,*train_data_],[prop, *posteriors_]
    elif algo == 'SNPE':
        inf = infer.SNPE(**kwargs, convert_to_T=None)
        inf.generator.proposal = prop
        log_, train_data_, posteriors_ = inf.run(n_train=train,
                                              minibatch=minibatch,                                              
                                              n_rounds=len(train),
                                              epochs=epochs, 
                                              round_cl=round_cl)        
        log,train_data,posteriors = [None,*log_],[None,*train_data_],[prop, *posteriors_]
    elif algo == 'kSNPE':
        inf = infer.SNPE(**kwargs, convert_to_T=None)
        inf.generator.proposal = prop
        log_, train_data_, posteriors_ = inf.run(n_train=train,
                                              minibatch=minibatch,                                              
                                              n_rounds=len(train),
                                              epochs=epochs, 
                                              kernel_loss='x_kl', 
                                              cbk_feature_layer=cbk_feature_layer,                                              
                                              round_cl=round_cl)      
        log,train_data,posteriors = [None,*log_],[None,*train_data_],[prop, *posteriors_]
    elif algo == 'Algo1':
        inf = infer.SNPE(**kwargs, convert_to_T=None)
        log, train_data, posteriors = inf.run(n_train=train,
                                              minibatch=minibatch,                                              
                                              n_rounds=len(train),
                                              epochs=epochs, 
                                              round_cl=round_cl)   

    else:
        raise ValueError
        
    try:
        posterior = inf.predict(obs)
    except:
        posterior = None
    
    
    
    print('algorithm: ', algo)

    try:
        w = train_data[-1][2]
        w /= w.sum()
        print('ESS', 1/np.sum(w**2))

        plt.subplot(1,3,1)
        xx = np.linspace(3., 5., 100)
        plt.plot(xx, posteriors[-1].eval(xx,ii=[0],log=False), 'k', linewidth=2.)
        plt.plot(thetao[0], 1., 'r*')

        plt.subplot(1,3,2)
        xx = np.linspace(0, 0.8, 100)
        #plt.plot(xx, lognorm.pdf(x=xx, 
        #                         s=np.sqrt(posteriors[-1].xs[0].S[1,1]), 
        #                         scale=np.exp(-posteriors[-1].xs[0].m[1])), 'k', linewidth=2.)
        plt.plot(xx, posteriors[-1].eval(xx,ii=[1],log=False), 'k', linewidth=2.)
        plt.plot(thetao[1], 1., 'r*')


        plt.subplot(1,3,3)
        xx = np.linspace(4., 20., 200)
        plt.plot(xx, posteriors[-1].eval(xx,ii=[2],log=False), 'k', linewidth=2.)
        plt.plot(thetao[2], 0.5, 'r*')        

        #plt.plot(xx, lognorm.pdf(x=xx, 
        #                         s=np.sqrt(posteriors[-1].xs[0].S[2,2]), 
        #                         scale=np.exp(-posteriors[-1].xs[0].m[2])), 'k', linewidth=2.)
        #plt.plot(np.exp(-lthetao[2]), 0.5, 'r*')        

        plt.show()
        
    except:
        pass
        
    return {'algo': algo,
            'rounds': rounds,
            'seed': seed,
            'posterior': posterior,
            'generator': inf.generator,
            'log': log,
            'obs' : obs_stats, 
            'train_data': train_data,
            'posteriors': posteriors,
            'network' : inf.network,
            'stats_std' : inf.stats_std,
            'stats_mean' : inf.stats_mean,
            'cbk_feature_layer' : cbk_feature_layer
           }
        


In [ ]:

rounds = 1
n_train = [10000]

seeds = range(90, 110)
all_outs_SNPE, all_outs_kSNPE, all_outs_CDELFI = [], [], []
all_outs_large = []
for seed in seeds:
        
    #out = run_panel_c(algo='CDELFI', rounds=rounds, seed=seed, n_train=n_train)
    out = None
    all_outs_CDELFI.append(out)

    out_SNPE = run_panel_c(algo='SNPE', rounds=rounds, seed=seed, n_train=n_train)
    all_outs_SNPE.append(out_SNPE)        

    out_kSNPE = run_panel_c(algo='kSNPE', rounds=rounds, seed=seed, n_train=n_train)
    all_outs_kSNPE.append(out_kSNPE)
    
    out_comp = run_panel_c(algo='Algo1', rounds=1, seed=seed, n_train=[np.sum(n_train)])
    all_outs_large.append(out_comp)       


In [ ]:
rounds = 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(14,rounds*4))

data_path = 'results/'
tmp = np.load(data_path + 'Ricker_rejABC_xoGutmann_1e8simulations_results.npy')[()]
th_post = tmp['samples']
iws = np.ones(th_post.shape[0])/th_post.shape[0]
del tmp


fail_count_CDELFI = 0


plt.subplot(rounds,3,1)        
plt.plot([3., 5], [0.5, 0.5], 'k--')
plt.subplot(rounds,3,2)        
plt.plot([0., 0.8], [1.25, 1.25], 'k--')
plt.subplot(rounds,3,3)        
plt.plot([4., 20.], [1/16., 1/16.], 'k--')
plt.plot(0., 0., ':', color=np.array([0.3, 0.3, 0.3])) 
plt.plot(0., 0., 'r', linewidth=2.)
plt.plot(0., 0., 'g', linewidth=2.)

for i in range(len(seeds)):
    
    seed = seeds[i]

    #out = all_outs_CDELFI[i]
    out_SNPE = all_outs_SNPE[i]
    out_kSNPE = all_outs_kSNPE[i]
    
    xx = np.linspace(-1.5, 1.5, 200).reshape(-1,1)

    broke_flag = False
    for r in range(rounds):
        if rounds > len(n_train) and r < rounds-1:
            r_ = 0
        elif rounds > len(n_train) and r == rounds-1:
            r_ = len(n_train)-1
        else: 
            r_ = r

        plt.subplot(rounds,3,1 + r*3)        
        xx = np.linspace(3., 5., 100)        
        mkrs = ['-', '--'] if r == 0 else ['-', '-']
        plt.plot([thetao[0],thetao[0]], [0,3], ':', color=np.array([0.3, 0.3, 0.3]))        
        plt.hist(th_post[:,0], bins=np.concatenate((xx[::3], np.atleast_1d(xx[-1]))), weights=iws, normed=True, facecolor=np.array([80, 160, 220])/256)
        for out_, clr, lst in zip([all_outs_SNPE[i],all_outs_kSNPE[i]], ['r', 'g'], mkrs):
            posterior = out_['posteriors'][r]
            try:
                plt.plot(xx, posterior.eval(xx,ii=[0],log=False), lst, color=clr, linewidth=2.)
            except:
                pass
        plt.axis([3., 5., 0, 3.])
        plt.yticks([])
        plt.xticks([3, 4, 5])
        plt.tick_params(top='off', left='off', right='off', labelleft='off', labelbottom='on')
        plt.ylabel('round #' + str(r+1) + '(N=' + str(n_train[r_]) + ')')
        if r == rounds-1:
            plt.xlabel('log r')
        
        
        plt.subplot(rounds,3,2 + r*3)        
        xx = np.linspace(0., 0.8, 100)        
        mkrs = ['-', '--'] if r == 0 else ['-', '-']
        plt.plot([thetao[1],thetao[1]], [0,6.], ':', color=np.array([0.3, 0.3, 0.3]))        
        plt.hist(th_post[:,1], bins=np.concatenate((xx[::3], np.atleast_1d(xx[-1]))), weights=iws, normed=True, facecolor=np.array([80, 160, 220])/256)
        for out_, clr, lst in zip([all_outs_SNPE[i],all_outs_kSNPE[i]], ['r', 'g'], mkrs):
            posterior = out_['posteriors'][r]
            try:
                #plt.plot(xx, lognorm.pdf(x=xx, 
                #                         s=np.sqrt(posterior.xs[0].S[1,1]), 
                #                         scale=np.exp(-posterior.xs[0].m[1])), 
                #         lst, color=clr, linewidth=2.)            
                plt.plot(xx, posterior.eval(xx.reshape(-1,1),ii=[1],log=False), lst, color=clr, linewidth=2.)
            except: 
                pass
        plt.axis([0., 0.8, 0, 3.])
        plt.yticks([])
        plt.xticks([0, 0.4, 0.8])
        plt.tick_params(top='off', left='off', right='off', labelleft='off', labelbottom='on')
        if r == rounds-1:
            plt.xlabel('sigma')

        
        plt.subplot(rounds,3,3 + r*3)        
        xx = np.linspace(4., 20., 100)        
        mkrs = ['-', '--'] if r == 0 else ['-', '-']
        plt.plot([thetao[2],thetao[2]], [0,1.], ':', color=np.array([0.3, 0.3, 0.3]))        
        plt.hist(th_post[:,2], bins=np.concatenate((xx[::3], np.atleast_1d(xx[-1]))), weights=iws, normed=True, facecolor=np.array([80, 160, 220])/256)
        for out_, clr, lst in zip([all_outs_SNPE[i],all_outs_kSNPE[i]], ['r', 'g'], mkrs):
            posterior = out_['posteriors'][r]
            try:
                #plt.plot(xx, lognorm.pdf(x=xx, 
                #                         s=np.sqrt(posterior.xs[0].S[2,2]), 
                #                         scale=np.exp(-posterior.xs[0].m[2])), 
                #        lst, color=clr, linewidth=2.)  
                plt.plot(xx, posterior.eval(xx,ii=[2],log=False), lst, color=clr, linewidth=2.)
                
            except:
                pass
        plt.axis([4., 20, 0, 0.8])
        plt.yticks([])
        plt.xticks([4, 12, 20])
        plt.tick_params(top='off', left='off', right='off', labelleft='off', labelbottom='on')
        if r == rounds-1:
            plt.xlabel('phi')

plt.subplot(rounds,3,3)
plt.legend(['prior', 'ground-truth', 'SNPE', 'kSNPE'], loc=1)

plt.suptitle('Ricker_20seeds_2round_N11k_h20_20_3components_noSVI_externalProposal')
#plt.savefig('Ricker_20seeds_2round_N11k_h20_20_3components_noSVI_externalProposal.pdf')
    
plt.show()

In [ ]:
# [20, 20] network, 3 comopnents

In [ ]:
# [20,20] network, 2 components

In [ ]:
for i in range(len(seeds)):
    all_outs_kSNPE[i]['network'] = None
    all_outs_SNPE[i]['network'] = None    

In [ ]:

np.save('Ricker_20seeds_2round_N11k_h20_20_3components', 
        {'seeds' : seeds, 
         'n_train' : n_train,
         'rounds' : rounds, 
         
         'res_CDELFI' : all_outs_CDELFI,
         'res_kSNPE' : all_outs_kSNPE,
         'res_SNPE' : all_outs_SNPE        
    })
